# Google Gemini 2.5 with ADK (Agent Development Kit) and MCP (Model Context Protocol) Servers

Agent Development Kit (ADK) is a new modular framework for developing and deploying AI agents. ADK makes it easy to get started with simple agents powered by Gemini models and Google AI tools while providing the control and structure needed for more complex agent architectures and orchestration.

Gemini models can be used with MCP server using its native tool calling capabilities. MCP, or Model Context Protocol, is an open standard introduced by Anthropic designed to standardize how AI models like Gemini interact with external tools and data sources. Instead of requiring custom integrations for each tool, MCP provides a structured way for models to access context, such as functions (tools), data sources (resources), or pre-defined prompts. This allows AI agents to securely and efficiently connect with real-world systems and workflows.

MCP server expose their tools via JSON schema definitions, which can be converted to Gemini compatible OpenAPI schema definitions. This allows you to easily use MCP server with Gemini models, below you will example on how to implement this. 

You can learn more about Google Search integration with Gemini here:
- [https://ai.google.dev/gemini-api/docs/function-calling?lang=python](https://ai.google.dev/gemini-api/docs/function-calling?lang=python&example=weather)
- [https://google.github.io/adk-docs/tools/mcp-tools/](https://google.github.io/adk-docs/tools/mcp-tools/)

In [ ]:
%pip install google-adk --upgrade

In [ ]:
from datetime import datetime
from google.genai import types
from google.adk import Agent, Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset,StdioConnectionParams, StdioServerParameters

async def mcp_agent():
        # Step 1: Connect to MCP Server and Get Tools 
        tools = McpToolset(
            connection_params=StdioConnectionParams(
                server_params = StdioServerParameters(
                    command='npx',
                    args=[
                        "-y",  # Argument for npx to auto-confirm install
                        "@philschmid/weather-mcp",
                    ],
                ),
            ),
        )
        # Step 2: Define the Agent 
        root_agent = Agent(
            model="gemini-2.5-flash",
            name='weather_assistant',
            instruction='Can read the weather in a specific city',
            tools=[tools], 
        )

        # Step 3: Setup Session, Runner, and Execute 
        session_service = InMemorySessionService()
        session = await session_service.create_session(app_name='weather_app', user_id='john', session_id="s-john")
        runner = Runner(agent=root_agent, app_name='weather_app', session_service=session_service)

        # Runner is responsible for executing the agent
        runner = Runner(
            app_name='weather_app',
            agent=root_agent,
            session_service=session_service,
        )

        # Step 4: Run the agent 
        prompt = f"What is the weather in Berlin tomorrow, {datetime.now().strftime('%Y-%m-%d')}?"
        user_content = types.Content(role='user', parts=[types.Part(text=prompt)])

        events_async = runner.run_async(
            session_id=session.id, user_id=session.user_id, new_message=user_content
        )

        async for event in events_async:
            if event.content.parts[0].function_call:
                print(f"Function call: {event.content.parts[0].function_call.args}")
            elif event.content.parts[0].function_response:
                print(f"Function response: {event.content.parts[0].function_response.response}")
            else:
                print(event.content.parts[0].text)
                
await mcp_agent()

/Users/philschmid/projects/personal/gemini-samples/.venv/lib/python3.13/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


Function call: {'location': 'Berlin, Germany', 'date': '2025-09-21'}


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16759e490>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x1678c4170>, 1055386.773307625)])']
connector: <aiohttp.connector.TCPConnector object at 0x16759de50>
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


Function response: {'result': CallToolResult(meta=None, content=[TextContent(type='text', text='{"2025-09-21T00:00":20.4,"2025-09-21T01:00":19.9,"2025-09-21T02:00":19.2,"2025-09-21T03:00":18.3,"2025-09-21T04:00":18.7,"2025-09-21T05:00":18.5,"2025-09-21T06:00":17.6,"2025-09-21T07:00":18.7,"2025-09-21T08:00":20.6,"2025-09-21T09:00":23.5,"2025-09-21T10:00":25.7,"2025-09-21T11:00":26.4,"2025-09-21T12:00":26.1,"2025-09-21T13:00":24.7,"2025-09-21T14:00":24,"2025-09-21T15:00":22.4,"2025-09-21T16:00":20.8,"2025-09-21T17:00":19.6,"2025-09-21T18:00":17.7,"2025-09-21T19:00":16.9,"2025-09-21T20:00":15.8,"2025-09-21T21:00":15,"2025-09-21T22:00":14.2,"2025-09-21T23:00":13.6}', annotations=None, meta=None)], structuredContent=None, isError=False)}


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16759e5d0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x1678c4470>, 1055389.886287083)])']
connector: <aiohttp.connector.TCPConnector object at 0x16759fd90>


Here is the weather forecast for Berlin on 2025-09-21:
00:00: 20.4°C
01:00: 19.9°C
02:00: 19.2°C
03:00: 18.3°C
04:00: 18.7°C
05:00: 18.5°C
06:00: 17.6°C
07:00: 18.7°C
08:00: 20.6°C
09:00: 23.5°C
10:00: 25.7°C
11:00: 26.4°C
12:00: 26.1°C
13:00: 24.7°C
14:00: 24.0°C
15:00: 22.4°C
16:00: 20.8°C
17:00: 19.6°C
18:00: 17.7°C
19:00: 16.9°C
20:00: 15.8°C
21:00: 15.0°C
22:00: 14.2°C
23:00: 13.6°C
